In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import r2_score 
from alpaca.uncertainty_estimator import build_estimator
from alpaca.model.mlp import MLP 
from alpaca.dataloader.builder import build_dataset
from alpaca.analysis.metrics import ndcg

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# Load dataset
dataset = build_dataset('kin8nm', val_split=1_000)
x_train, y_train = dataset.dataset('train')
x_val, y_val = dataset.dataset('val')
x_train.shape, y_val.shape
train_ds = TensorDataset(torch.DoubleTensor(x_train), torch.DoubleTensor(y_train))
val_ds = TensorDataset(torch.DoubleTensor(x_val), torch.DoubleTensor(y_val))
train_loader = DataLoader(train_ds, batch_size=512)
val_loader = DataLoader(val_ds, batch_size=512)


In [3]:
# Train model
layers = (8, 256, 128, 64, 1)
model = MLP(layers)
model.to(device)

MLP(
  (fc0): Linear(in_features=8, out_features=256, bias=True)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
  (criterion): MSELoss()
)

In [4]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

model.train()
for epochs in range(10):
    for x_batch, y_batch in train_loader: # Train for one epoch
        predictions = model(x_batch.to(device))
        loss = criterion(predictions, y_batch.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
print('Train loss on last batch', loss.item())


Train loss on last batch 0.02297538784192168


In [5]:
# Check model effectiveness 
model.eval()
x_batch, y_batch = next(iter(val_loader))
predictions = model(x_batch.to(device)).detach().cpu().numpy()
print('R2', r2_score(predictions, y_batch))

R2 0.5659766691048308


In [6]:
# Calculate uncertainty estimation
estimator = build_estimator("mcdue_masked", model, dropout_mask='mc_dropout')
estimations = estimator.estimate(x_batch.to(device))
print(estimations[:10])

[0.26284338 0.28408811 0.2061776  0.35274335 0.17020405 0.45938496
 0.28834275 0.3821859  0.23246293 0.20719471]


In [7]:
# Calculate NDCG score for the uncertainty
errors = np.abs(estimations - y_batch.reshape((-1)).numpy()) 
score = ndcg(np.array(errors), estimations)
print("Quality score is ", score)

Quality score is  0.80131713141473
